In [1]:
import sys
sys.path.append('..')
print(sys.path)

['c:\\Users\\terc.vinicius\\covid19datascience\\notebooks', 'c:\\Users\\terc.vinicius\\.vscode\\extensions\\ms-toolsai.jupyter-2021.2.576440691\\pythonFiles\\vscode_datascience_helpers\\..\\.does-not-exist', 'c:\\Users\\terc.vinicius\\.vscode\\extensions\\ms-toolsai.jupyter-2021.2.576440691\\pythonFiles', 'c:\\Users\\terc.vinicius\\.vscode\\extensions\\ms-toolsai.jupyter-2021.2.576440691\\pythonFiles\\lib\\python', 'C:\\ProgramData\\Anaconda3\\python38.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', '', 'C:\\Users\\terc.vinicius\\AppData\\Roaming\\Python\\Python38\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\terc.vinicius\\.ipython', '..']


In [2]:
from sys import exit
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from bulletin.commom import static, normalize
from bulletin.data.casos_confirmados import CasosConfirmados

from bulletin.commom.utils import Timer, auto_fit_columns
pd.set_option('display.max_columns', None)

meses = ['janeiro','fevereiro','marco','abril','maio','junho','julho','agosto','setembro','outubro','novembro','dezembro']
pd.set_option("display.precision", 5)

In [9]:
# csv_geral = pd.read_csv('geral.csv', sep=';', converters={'IDADE_ORIGINAL': lambda x: normalize.normalize_number(x,fill=0)}, parse_dates=['DATA_DIAGNOSTICO','DATA_OBITO'], date_parser=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
# csv_geral.columns = [ normalize.normalize_labels(x) for x in csv_geral.columns ]
# casos = csv_geral.copy()
cc = CasosConfirmados()
# cc.update()
casos = cc.get_casos()
casos.columns = [ normalize.normalize_labels(x) for x in casos.columns ]
casos = casos.rename(columns={'idade':'idade_original','dt_diag':'data_diagnostico','ibge7':'ibge_res_pr'})
cc.shape()
casos = casos.loc[casos['ibge_res_pr'].notna()].copy()

In [10]:
faixa_etaria = [60,70,80,90,100,np.inf]
faixa_etaria_labels = ['<=59','60-69','70-79','80-89','90-99','>=100']
faixa_etaria_index = dict(enumerate(faixa_etaria_labels))
index_casos = np.digitize(casos['idade_original'], faixa_etaria, right=True)

index_casos

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [11]:
casos['mes_caso'] = casos.apply(lambda row: row['data_diagnostico'].strftime('%m'), axis=1)
casos['ano_caso'] = casos.apply(lambda row: row['data_diagnostico'].strftime('%Y'), axis=1)
casos['faixa_etaria'] = [ faixa_etaria_labels[index] for index in index_casos ]

In [12]:
obitos = casos.loc[ casos['data_obito'].notnull() ].copy()
obitos['mes_obito'] = obitos.apply(lambda row: row['data_obito'].strftime('%m'), axis=1)
obitos['ano_obito'] = obitos.apply(lambda row: row['data_obito'].strftime('%Y'), axis=1)

,ordem,identificacao,ibge_res_pr,rs_res_pr,nome,sexo,idade_original,mun_resid,mun_atend,rs,laboratorio,data_diagnostico,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_dt_diag,mes_caso,ano_caso,faixa_etaria,mes_obito,ano_obito
108,109,10853,4109401,5.0,FERNANDO JOSE COSTA,M,57,GUARAPUAVA,GUARAPUAVA,05,LACEN,2020-03-26,2020-03-27,2020-03-08,SIM,2020-06-15,NaN,FERNANDOJOSECOSTA57GUARAPUAVA,FERNANDOJOSECOSTA56GUARAPUAVA,FERNANDOJOSECOSTA58GUARAPUAVA,FERNANDOJOSECOSTA57GUARAPUAVA,FERNANDOJOSECOSTA56GUARAPUAVA,FERNANDOJOSECOSTA58GUARAPUAVA,FERNANDOJOSECOSTA26032020,03,2020,<=59,06,2020
111,112,6249,4106902,2.0,CLODOALDO PEDRO BIASOTTO,M,72,CURITIBA,CURITIBA,02,LACEN,2020-03-27,2020-03-27,NaT,SIM,2020-04-07,NaN,CLODOALDOPEDROBIASOTTO72CURITIBA,CLODOALDOPEDROBIASOTTO71CURITIBA,CLODOALDOPEDROBIASOTTO73CURITIBA,CLODOALDOPEDROBIASOTTO72CURITIBA,CLODOALDOPEDROBIASOTTO71CURITIBA,CLODOALDOPEDROBIASOTTO73CURITIBA,CLODOALDOPEDROBIASOTTO27032020,03,2020,70-79,04,2020
114,115,27696,4115200,15.0,ROSANGELA ANTUNES MACHADO DA,F,54,MARINGA,SARANDI,15,LACEN,2020-03-27,2020-03-27,2020-03-14,SIM,2020-03-25,NaN,ROSANGELAANTUNESMACHADODA54MARINGA,ROSANGELAANTUNESMACHADODA53MARINGA,ROSANGELAANTUNESMACHADODA55MARINGA,ROSANGELAANTUNESMACHADODA54SARANDI,ROSANGELAANTUNESMACHADODA53SARANDI,ROSANGELAANTUNESMACHADODA55SARANDI,ROSANGELAANTUNESMACHADODA27032020,03,2020,<=59,03,2020
115,116,4062,4115200,15.0,BELCHIOR MARTINS,M,84,MARINGA,MARINGA,15,LACEN,2020-03-27,2020-03-27,2020-03-15,SIM,2020-03-26,NaN,BELCHIORMARTINS84MARINGA,BELCHIORMARTINS83MARINGA,BELCHIORMARTINS85MARINGA,BELCHIORMARTINS84MARINGA,BELCHIORMARTINS83MARINGA,BELCHIORMARTINS85MARINGA,BELCHIORMARTINS27032020,03,2020,80-89,03,2020
116,117,11830,4110805,11.0,GERSON LACERDA DE SOUZA,M,51,IRETAMA,IRETAMA,11,LACEN,2020-03-27,2020-03-27,NaT,SIM,2020-04-10,NaN,GERSONLACERDADESOUZA51IRETAMA,GERSONLACERDADESOUZA50IRETAMA,GERSONLACERDADESOUZA52IRETAMA,GERSONLACERDADESOUZA51IRETAMA,GERSONLACERDADESOUZA50IRETAMA,GERSONLACERDADESOUZA52IRETAMA,GERSONLACERDADESOUZA27032020,03,2020,<=59,04,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909183,909196,988239,4119152,2.0,REYNALDO CANTARELLA,M,72,PINHAIS,CAMPO LARGO,02,VIRUS RESPIRATORIO,2021-04-15,2021-04-19,NaT,SIM,2021-04-19,NaN,REYNALDOCANTARELLA72PINHAIS,REYNALDOCANTARELLA71PINHAIS,REYNALDOCANTARELLA73PINHAIS,REYNALDOCANTARELLA72CAMPOLARGO,REYNALDOCANTARELLA71CAMPOLARGO,REYNALDOCANTARELLA73CAMPOLARGO,REYNALDOCANTARELLA15042021,04,2021,70-79,04,2021
909323,909336,988379,4101408,16.0,SANDOVAL VITALINO,M,68,APUCARANA,APUCARANA,16,COVID-19 BIOLOGIA MOLECULAR,2021-03-26,2021-04-19,NaT,SIM,2021-04-15,NaN,SANDOVALVITALINO68APUCARANA,SANDOVALVITALINO67APUCARANA,SANDOVALVITALINO69APUCARANA,SANDOVALVITALINO68APUCARANA,SANDOVALVITALINO67APUCARANA,SANDOVALVITALINO69APUCARANA,SANDOVALVITALINO26032021,03,2021,60-69,04,2021
909490,909503,988546,4128559,10.0,TEREZINHA BEIRA MAGALHAES,F,62,VERA CRUZ DO OESTE,VERA CRUZ DO OESTE,10,VIRUS RESPIRATORIO,2021-04-16,2021-04-19,2021-04-12,SIM,2021-04-18,NaN,TEREZINHABEIRAMAGALHAES62VERACRUZDOOESTE,TEREZINHABEIRAMAGALHAES61VERACRUZDOOESTE,TEREZINHABEIRAMAGALHAES63VERACRUZDOOESTE,TEREZINHABEIRAMAGALHAES62VERACRUZDOOESTE,TEREZINHABEIRAMAGALHAES61VERACRUZDOOESTE,TEREZINHABEIRAMAGALHAES63VERACRUZDOOESTE,TEREZINHABEIRAMAGALHAES16042021,04,2021,60-69,04,2021
909656,909669,988712,4106902,2.0,WESLEY DIEGO DA SILVA,M,32,CURITIBA,CURITIBA,02,COVID-19 BIOLOGIA MOLECULAR,2021-04-04,2021-04-19,2021-04-03,SIM,2021-04-14,NaN,WESLEYDIEGODASILVA32CURITIBA,WESLEYDIEGODASILVA31CURITIBA,WESLEYDIEGODASILVA33CURITIBA,WESLEYDIEGODASILVA32CURITIBA,WESLEYDIEGODASILVA31CURITIBA,WESLEYDIEGODASILVA33CURITIBA,WESLEYDIEGODASILVA04042021,04,2021,<=59,04,2021


In [13]:
faixa_etaria_casos = casos.groupby(by=['ano_caso','mes_caso','faixa_etaria'])[['sexo','ibge_res_pr']].count().rename(columns={'sexo':'0','ibge_res_pr':'1'})
faixa_etaria_casos = faixa_etaria_casos.unstack().fillna(0).astype(int)
faixa_etaria_casos = faixa_etaria_casos.swaplevel(1,0,1).sort_index(1).rename(columns={'0':'qtde','1':'%'})

faixa_etaria_casos.columns = pd.MultiIndex.from_tuples(faixa_etaria_casos.columns)
faixa_etaria_casos['total'] = faixa_etaria_casos.xs('qtde',level=1, axis=1).sum(axis=1)


# for ano in faixa_etaria_casos.index.get_level_values(0).unique():
#     faixa_etaria_casos = faixa_etaria_casos.append(
#         pd.Series(data=faixa_etaria_casos.loc[ano].sum(0), name=(ano,'total'))
#     )

# faixa_etaria_casos = faixa_etaria_casos.append(
#         pd.Series(data=faixa_etaria_casos.xs('total', level=1).sum(0), name=('total_geral',''))
# )

faixa_etaria_casos = faixa_etaria_casos.loc[faixa_etaria_casos.sum(1)>0]
faixa_etaria_casos = faixa_etaria_casos.sort_index()

for ano_mes in faixa_etaria_casos.index:
    for faixa_etaria, tipo in faixa_etaria_casos.columns:
        if tipo == '%':
            faixa_etaria_casos.loc[ano_mes,(faixa_etaria,'%')] = faixa_etaria_casos.loc[ano_mes,(faixa_etaria,'%')] / faixa_etaria_casos.loc[ano_mes,'total'].values
            
faixa_etaria_casos

60-69          70-79          80-89          90-99  \
                    qtde        %  qtde        %  qtde        %  qtde   
ano_caso mes_caso                                                       
2020     03           44  0.11924    13  0.03523     2  0.00542     0   
         04          173  0.08427   100  0.04871    61  0.02971    11   
         05          342  0.06503   184  0.03499    73  0.01388     7   
         06         1851  0.06683   881  0.03181   363  0.01311    47   
         07         4742  0.07062  2230  0.03321   921  0.01372   169   
         08         4706  0.06984  2228  0.03306   920  0.01365   180   
         09         4059  0.07107  1888  0.03306   725  0.01269   147   
         10         2786  0.07291  1271  0.03326   527  0.01379    78   
         11         6833  0.07096  2920  0.03033   986  0.01024   155   
         12        10344  0.07995  4712  0.03642  1688  0.01305   254   
2021     01         9381  0.08187  4471  0.03902  1631  0.01423   252   
         02         8529  0.08431  4047  0.04001  1411  0.01395   193   
         03        13040  0.08524  6157  0.04025  2123  0.01388   309   
         04         3816  0.08583  1825  0.04105   536  0.01206    72   

                              <=59          >=100            total  
                         %    qtde        %  qtde        %          
ano_caso mes_caso                                                   
2020     03        0.00000     310  0.84011     0  0.00000     369  
         04        0.00536    1708  0.83195     0  0.00000    2053  
         05        0.00133    4653  0.88477     0  0.00000    5259  
         06        0.00170   24555  0.88649     2  0.00007   27699  
         07        0.00252   59082  0.87985     6  0.00009   67150  
         08        0.00267   59347  0.88070     5  0.00007   67386  
         09        0.00257   50291  0.88052     5  0.00009   57115  
         10        0.00204   33548  0.87792     3  0.00008   38213  
         11        0.00161   85388  0.88681     5  0.00005   96287  
         12        0.00196  112366  0.86848    19  0.00015  129383  
2021     01        0.00220   98844  0.86262     7  0.00006  114586  
         02        0.00191   86971  0.85975     8  0.00008  101159  
         03        0.00202  131330  0.85852    14  0.00009  152973  
         04        0.00162   38210  0.85940     2  0.00004   44461

In [14]:
faixa_etaria_obitos = obitos.groupby(by=['ano_obito','mes_obito','faixa_etaria'])[['sexo','ibge_res_pr']].count().rename(columns={'sexo':'0','ibge_res_pr':'1'})
faixa_etaria_obitos = faixa_etaria_obitos.unstack().fillna(0).astype(int)
faixa_etaria_obitos = faixa_etaria_obitos.swaplevel(1,0,1).sort_index(1).rename(columns={'0':'qtde','1':'%'})

faixa_etaria_obitos.columns = pd.MultiIndex.from_tuples(faixa_etaria_obitos.columns)
faixa_etaria_obitos['total'] = faixa_etaria_obitos.xs('qtde',level=1, axis=1).sum(axis=1)


# for ano in faixa_etaria_obitos.index.get_level_values(0).unique():
#     faixa_etaria_obitos = faixa_etaria_obitos.append(
#         pd.Series(data=faixa_etaria_obitos.loc[ano].sum(0), name=(ano,'total'))
#     )

# faixa_etaria_obitos = faixa_etaria_obitos.append(
#         pd.Series(data=faixa_etaria_obitos.xs('total', level=1).sum(0), name=('total_geral',''))
# )

faixa_etaria_obitos = faixa_etaria_obitos.loc[faixa_etaria_obitos.sum(1)>0]
faixa_etaria_obitos = faixa_etaria_obitos.sort_index()

for ano_mes in faixa_etaria_obitos.index:
    for faixa_etaria, tipo in faixa_etaria_obitos.columns:
        if tipo == '%':
            faixa_etaria_obitos.loc[ano_mes,(faixa_etaria,'%')] = faixa_etaria_obitos.loc[ano_mes,(faixa_etaria,'%')] / faixa_etaria_obitos.loc[ano_mes,'total'].values
            
faixa_etaria_obitos

60-69          70-79          80-89          90-99  \
                     qtde        %  qtde        %  qtde        %  qtde   
ano_obito mes_obito                                                      
2020      03            1  0.16667     1  0.16667     1  0.16667     0   
          04           15  0.14851    21  0.20792    21  0.20792    10   
          05           33  0.27731    28  0.23529    22  0.18487     2   
          06          136  0.22332   159  0.26108   139  0.22824    18   
          07          331  0.23277   374  0.26301   254  0.17862    59   
          08          395  0.25337   405  0.25978   286  0.18345    64   
          09          306  0.24757   334  0.27023   223  0.18042    69   
          10          214  0.26355   217  0.26724   164  0.20197    36   
          11          271  0.26700   274  0.26995   199  0.19606    38   
          12          554  0.25589   605  0.27945   401  0.18522    78   
2021      01          489  0.26634   504  0.27451   345  0.18791    73   
          02          473  0.25609   515  0.27883   299  0.16188    65   
          03         1473  0.25874  1362  0.23924   749  0.13157   132   
          04          543  0.29022   444  0.23731   181  0.09674    26   

                              <=59          >=100          total  
                           %  qtde        %  qtde        %        
ano_obito mes_obito                                               
2020      03         0.00000     3  0.50000     0  0.00000     6  
          04         0.09901    34  0.33663     0  0.00000   101  
          05         0.01681    34  0.28571     0  0.00000   119  
          06         0.02956   157  0.25780     0  0.00000   609  
          07         0.04149   401  0.28200     3  0.00211  1422  
          08         0.04105   407  0.26106     2  0.00128  1559  
          09         0.05583   301  0.24353     3  0.00243  1236  
          10         0.04433   181  0.22291     0  0.00000   812  
          11         0.03744   232  0.22857     1  0.00099  1015  
          12         0.03603   522  0.24111     5  0.00231  2165  
2021      01         0.03976   423  0.23039     2  0.00109  1836  
          02         0.03519   492  0.26638     3  0.00162  1847  
          03         0.02319  1971  0.34621     6  0.00105  5693  
          04         0.01390   676  0.36130     1  0.00053  1871

In [15]:
def set_format_cols(worksheet,cols,format):
    for col in cols:
        worksheet.set_column(f"{col}:{col}", None, format)


writer = pd.ExcelWriter("faixa_etaria_por_mes_raw.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

workbook = writer.book

format1 = workbook.add_format({'num_format': '#.##0'})
# format2 = workbook.add_format({'num_format': '0,00%'})

faixa_etaria_casos.to_excel(writer,sheet_name=f"faixa_etaria_casos")
worksheet = writer.sheets[f"faixa_etaria_casos"]
# set_format_cols(worksheet,['C','E','G','I','K','M','O'],format1)
# set_format_cols(worksheet,['D','F','H','J','L','N'],format2)
auto_fit_columns(worksheet,faixa_etaria_casos)


faixa_etaria_obitos.to_excel(writer,sheet_name=f"faixa_etaria_obitos")
worksheet = writer.sheets[f"faixa_etaria_obitos"]
# set_format_cols(worksheet,['C','E','G','I','K','M','O'],format1)
# set_format_cols(worksheet,['D','F','H','J','L','N'],format2)
auto_fit_columns(worksheet,faixa_etaria_obitos)

writer.save()
writer.close()